In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from solve_network import palette
import yaml
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter

### Load results

In [ ]:
def load_configuration(config_path):
    """
    Load configuration settings from a YAML file.
    """
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config


config = load_configuration("../config.yaml")

run = "manuscript-dashboard1"  # run name from config.yaml

In [ ]:
datacenters = config["ci"]["datacenters"]  # pair name from config.yaml
locations = list(datacenters.keys())
names = list(datacenters.values())

scaling = int(config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H

participation = config["scenario"]["participation"]
palette_techs = palette("p4")
year = 2025


(
    clean_techs,
    storage_techs,
    storage_charge_techs,
    storage_discharge_techs,
) = palette_techs

# renaming technologies for plotting
clean_chargers = [tech.replace(" ", "_") for tech in storage_charge_techs]
clean_dischargers = [tech.replace(" ", "_") for tech in storage_discharge_techs]


def tech_names(base_names, year):
    return [f"{name.replace(' ', '_')}-{year}" for name in base_names]


# expected technology names with year
exp_generators = tech_names(["offwind-ac", "offwind-dc", "onwind", "solar"], year)
exp_links = tech_names(["OCGT"], year)
exp_chargers = tech_names(["battery charger", "H2 Electrolysis"], year)
exp_dischargers = tech_names(["battery discharger", "H2 Fuel Cell"], year)

# Assign colors
tech_colors = config["tech_colors"]

# Rename mappings
rename_ci_cost = pd.Series(
    {
        "onwind": "onshore wind",
        "solar": "solar PV",
        "grid": "grid imports",
        "revenue": "revenue",
        "battery_storage": "battery storage",
        "battery_inverter": "battery storage",
        "battery_charger": "battery storage",
        "ironair_storage": "ironair storage",
        "ironair_inverter": "ironair storage",
        "ironair_charger": "ironair storage",
        "hydrogen_storage": "hydrogen storage",
        "hydrogen_electrolysis": "hydrogen storage",
        "hydrogen_fuel_cell": "hydrogen storage",
        "adv_geothermal": "advanced dispatchable",
        "allam_ccs": "NG-Allam",
    }
)

rename_ci_capacity = pd.Series(
    {
        "onwind": "onshore wind",
        "solar": "solar PV",
        "battery_discharger": "battery storage",
        "battery_charger": "battery storage",
        "ironair_discharger": "ironair storage",
        "ironair_charger": "ironair storage",
        "H2_Fuel_Cell": "hydrogen fuel cell",
        "H2_Electrolysis": "hydrogen electrolysis",
        "adv_geothermal": "advanced dispatchable",
        "allam_ccs": "NG-Allam",
    }
)

preferred_order = pd.Index(
    [
        "advanced dispatchable",
        "NG-Allam",
        "Gas OC",
        "offshore wind",
        "onshore wind",
        "solar PV",
        "battery storage",
        "ironair storage",
        "hydrogen storage",
        "hydrogen electrolysis",
        "hydrogen fuel cell",
    ]
)

### Collect results

In [ ]:
import glob


def find_summary_files(run):
    path_pattern = f"../results/{run}/**/summary.csv"
    summary_files = glob.glob(path_pattern, recursive=True)
    return summary_files


summary_files = find_summary_files(run)

In [ ]:
# concatenate all summary files into a single dataframe
df = pd.concat(
    [pd.read_csv(f, index_col=0, header=[0, 1]) for f in summary_files],
    axis=1,
    keys=["-".join(f.split("/")[-3:-1]) for f in summary_files],
)

# df

In [ ]:
# drop country level
df.columns = df.columns.droplevel([2])

# select 5% scenario
df = df.xs("5", axis=1, level=1)
# df

### Plotting

In [ ]:
FONTSIZE = 14


def _format_label(label):
    # Replace "-" with a newline
    label = label.replace("-", "\n")

    # Update p4 to p3 (we don't need p3 scenario here)
    label = label.replace("p4", "p3")

    # nicer tick labels
    label = (
        label.replace("p1", "Tech-1").replace("p2", "Tech-2").replace("p3", "Tech-3")
    )
    label = (
        label.replace("cfe90", "CFE 90%")
        .replace("cfe98", "CFE 98%")
        .replace("cfe100", "CFE 100%")
    )

    return label


scenarios_to_plot = [
    "p1-cfe90",
    "p2-cfe90",
    "p4-cfe90",
    "p1-cfe98",
    "p2-cfe98",
    "p4-cfe98",
    "p1-cfe100",
    "p2-cfe100",
    "p4-cfe100",
]

In [ ]:
ldf = pd.concat(
    [
        df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_techs]].rename(
            {"ci_cap_" + t: t for t in clean_techs}
        ),
        df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_dischargers]].rename(
            {"ci_cap_" + t: t for t in clean_dischargers}
        ),
        df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_chargers]].rename(
            {"ci_cap_" + t: t for t in clean_chargers}
        ),
    ]
)

# Display discharger capacity for storage technologies with fixed P/E ratio
to_drop = ["battery_charger", "ironair_charger"]
ldf = ldf.drop(index=[row for row in to_drop if row in ldf.index])

# Drop rows with all values less than 0.1
to_drop = ldf.index[(ldf < 0.1).all(axis=1)]
ldf.drop(to_drop, inplace=True)

# Rename columns and indices
# rename_scen(ldf, level=0)
ldf.rename(index=rename_ci_capacity, level=0, inplace=True)

# Reorder and sort the final DataFrame
new_index = preferred_order.intersection(ldf.index).append(
    ldf.index.difference(preferred_order)
)

ldf = ldf.loc[new_index]

ldf = ldf.loc[:, scenarios_to_plot]

In [ ]:
def ci_capacity(ax, df, tech_colors, rename_ci_capacity, preferred_order):
    # Data Preparation
    ldf = pd.concat(
        [
            df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_techs]].rename(
                {"ci_cap_" + t: t for t in clean_techs}
            ),
            df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_dischargers]].rename(
                {"ci_cap_" + t: t for t in clean_dischargers}
            ),
            df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_chargers]].rename(
                {"ci_cap_" + t: t for t in clean_chargers}
            ),
        ]
    )

    # Display discharger capacity for storage technologies with fixed P/E ratio
    to_drop = ["battery_charger", "ironair_charger"]
    ldf = ldf.drop(index=[row for row in to_drop if row in ldf.index])

    # Drop rows with all values less than 0.1
    to_drop = ldf.index[(ldf < 0.1).all(axis=1)]
    ldf.drop(to_drop, inplace=True)

    # Rename columns and indices
    # rename_scen(ldf, level=0)
    ldf.rename(index=rename_ci_capacity, level=0, inplace=True)

    # Reorder and sort the final DataFrame
    new_index = preferred_order.intersection(ldf.index).append(
        ldf.index.difference(preferred_order)
    )

    ldf = ldf.loc[new_index]

    ldf = ldf.loc[:, scenarios_to_plot]

    # Plotting Enhancements
    # fig, ax = plt.subplots(figsize=(8, 6))

    if not ldf.empty:
        ldf.T.plot(
            kind="bar",
            stacked=True,
            ax=ax,
            color=tech_colors,
            width=0.65,
            edgecolor="black",
            linewidth=0.05,
        )

        ax.set_xticklabels(
            [_format_label(col) for col in ldf.columns.tolist()], fontsize=FONTSIZE
        )
        ax.grid(alpha=0.8, which="both", linestyle="--")
        ax.set_axisbelow(True)
        # ax.set_ylim([0, max(ldf.sum()) * 1.2])
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.0f"))
        ax.tick_params(axis="y", labelsize=FONTSIZE)
        ax.set_ylabel("C&I procurement [MW]", fontsize=14)
        ax.legend(loc="upper left", ncol=1, prop={"size": FONTSIZE})

        ci_demand = df.loc["ci_demand_total"][0] / 8760
        ax.axhline(y=ci_demand, color="black", linestyle="--")
        ax.text(
            1,
            ci_demand,
            f"C&I hourly demand",
            verticalalignment="bottom",
            horizontalalignment="right",
            color="black",
            fontsize=FONTSIZE,
        )

        ax.set_title(
            f"24/7 CFE portfolio capacity",
            fontsize=16,
            weight="bold",
        )

        plt.xticks(rotation=0)

    else:
        print("Dataframe to plot is empty")

    # fig.tight_layout()
    # fig.show()

In [ ]:
def ci_costandrev(ax, df, tech_colors, rename_ci_cost, preferred_order):
    techs = clean_techs + [
        "grid",
        "battery_storage",
        "battery_inverter",
        "ironair_storage",
        "ironair_inverter",
        "hydrogen_storage",
        "hydrogen_electrolysis",
        "hydrogen_fuel_cell",
    ]

    # Calculate Costs
    costs = (
        df.loc[["ci_cost_" + t.replace(" ", "_") for t in techs]]
        .rename({"ci_cost_" + t: t for t in techs})
        .multiply(1 / df.loc["ci_demand_total"], axis=1)
        .fillna(0)
    )

    costs.drop(costs.index[(costs < 0.1).all(axis=1)], inplace=True)

    revenues = -df.loc[["ci_average_revenue"]].rename({"ci_average_revenue": "revenue"})
    # ldf = pd.concat([costs, revenues])
    ldf = pd.concat([costs])

    ldf = ldf.groupby(rename_ci_cost).sum()

    # Reorder and sort the DataFrame
    new_index = preferred_order.intersection(ldf.index).append(
        ldf.index.difference(preferred_order)
    )

    ldf = ldf.loc[new_index]

    ldf = ldf.loc[:, scenarios_to_plot]

    # treat 0% participation case (NaN and inf to 0.)
    ldf = ldf.fillna(0)
    ldf = ldf.replace([float("inf"), -float("inf")], 0)

    # Plotting
    # fig, ax = plt.subplots(figsize=(8, 6))

    if not ldf.empty:
        ldf.T.plot(
            kind="bar",
            stacked=True,
            ax=ax,
            color=tech_colors,
            width=0.65,
            edgecolor="black",
            linewidth=0.05,
        )
        ax.set_xticklabels(
            [_format_label(col) for col in ldf.columns.tolist()],
            fontsize=FONTSIZE,
            rotation=0,
        )
        ax.grid(alpha=0.8, which="both", linestyle="--")
        ax.set_axisbelow(True)
        ax.set_ylabel(r"Average costs [€$\cdot$MWh$^{-1}$]", fontsize=14)
        ax.legend(loc="upper left", ncol=1, prop={"size": FONTSIZE})
        # ax.set_ylim(top=max(ldf.sum()) * 1.5)
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.0f"))
        ax.tick_params(axis="y", labelsize=FONTSIZE)

        ax.set_title(
            r"C&I procurement costs [€$\cdot$MWh$^{-1}$]",
            fontsize=16,
            weight="bold",
        )

    else:
        print("Dataframe to plot is empty")

    # # Save Plot
    # fig.tight_layout()
    # fig.show()

In [ ]:
def ci_emisrate(ax, df):
    ldf = df.loc["ci_emission_rate_true"] * 1e3  # convert to gCO2/kWh
    reference = ldf.at["p1-ref"]

    ldf = ldf.loc[scenarios_to_plot]

    # Plotting
    # fig, ax = plt.subplots(figsize=(8, 6))

    if not ldf.empty:
        ldf.plot(
            kind="bar",
            ax=ax,
            color="#33415c",
            width=0.65,
            edgecolor="black",
            linewidth=0.05,
        )

        ax.axhline(y=reference, color="black", linestyle="--")
        ax.text(
            0.9,
            reference - 12,
            f"w/o 24/7 CFE policy: {reference:.2f} gCO2/kWh",
            verticalalignment="bottom",
            horizontalalignment="right",
            transform=ax.get_yaxis_transform(),
            color="black",
            fontsize=FONTSIZE,
        )

        ax.set_xticklabels(
            [_format_label(col) for col in ldf.index.tolist()],
            rotation=0,
            fontsize=FONTSIZE,
        )

        ax.grid(alpha=0.8, which="both", linestyle="--")
        ax.set_axisbelow(True)
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.0f"))
        ax.tick_params(axis="y", labelsize=FONTSIZE)
        ax.set_ylabel(r"C&I emission rate [gCO$_2$$\cdot$kWh$^{-1}$]", fontsize=14)
        ax.set_title(
            f"Emission rate of C&I consumption",
            fontsize=16,
            weight="bold",
        )
    else:
        print("Dataframe to plot is empty")

### Dashboard

In [ ]:
def create_dashboard(
    df, tech_colors, rename_ci_capacity, rename_ci_cost, preferred_order
):

    fig = plt.figure(figsize=(15, 20))

    gs = gridspec.GridSpec(3, 1, hspace=0.2, wspace=0.0)

    ax_1 = plt.subplot(gs[1, :1])
    ci_capacity(
        ax_1,
        df,
        tech_colors,
        rename_ci_capacity,
        preferred_order,
    )

    ax_2 = plt.subplot(gs[2, :1])
    ci_costandrev(ax_2, df, tech_colors, rename_ci_cost, preferred_order)

    ax_3 = plt.subplot(gs[0, :1])
    ci_emisrate(ax_3, df)

    fig.text(0.1, 0.88, "a", ha="left", fontsize=20, fontweight="bold")
    fig.text(0.1, 0.61, "b", ha="left", fontsize=20, fontweight="bold")
    fig.text(0.1, 0.34, "c", ha="left", fontsize=20, fontweight="bold")

    fig.tight_layout()
    fig.savefig(
        "../manuscript/images/dashboard_247CFE.pdf",
        transparent=True,
        bbox_inches="tight",
    )


create_dashboard(df, tech_colors, rename_ci_capacity, rename_ci_cost, preferred_order)